Acestea sunt codurile folosite pentru crearea bazelor de antrenament:

- Pentru algoritmul CMMP (oferă un fișier .csv)

In [ ]:
def make_mel_spectrogram_df(directory):
    labels = []
    mel_specs = []
    for file in os.scandir(directory):
        y, sr = librosa.core.load(file)
        label = str(file).split('.')[0][11:]
        labels.append(label)
        spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=1024)
        spect = librosa.power_to_db(spect, ref=np.max)
        if spect.shape[1] != 660: spect.resize(128,660, refcheck=False)
        spect = spect.flatten()
        mel_specs.append(spect)
    mel_specs = np.array(mel_specs)
    labels = np.array(labels).reshape(1000,1)
    df = pd.DataFrame(np.hstack((mel_specs,labels)))
    
    return df


df = make_mel_spectrogram_df('./genres/all')
df.to_csv('./genres/genre_mel_specs.csv', index=False)

- Pentru algoritmul KNN (oferă un fișier .dat)

In [ ]:
directory = "genres/gen"
f = open("my.dat" ,'wb')
i = 0

for folder in os.listdir(directory):
    i+=1
    if i==11 :
        break   
    for file in os.listdir(directory+"/"+folder+"/"):  
        try:
          (rate,sig) = wav.read(directory+"/"+folder+"/"+file)
        except:
          continue
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)

f.close()